In [1]:
# Adapted from https://github.com/lubianat/ann/blob/main/prototypes/notebooks/scispacy_linking_via_umls.ipynb

import scispacy
import spacy
import pandas as pd
from wikidataintegrator import wdi_core
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from functools import lru_cache

import en_core_sci_sm
nlp = en_core_sci_sm.load()

/home/lubianat/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_sci_sm' (0.2.4) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)
linker = EntityLinker(resolve_abbreviations=True, name="umls")
nlp.add_pipe(linker)


/home/lubianat/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/lubianat/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_semantic_type_tree.tsv not found in cache, downloading to /tmp/tmpfmm6z43j
Finished download, copying /tmp/tmpfmm6z43j to cache at /home/lubianat/.scispacy/datasets/21a1012c532c3a431d60895c509f5b4d45b0f8966c4178b892190a302b21836f.330707f4efe774134872b9f77f0e3208c1d30f50800b3b39a6b8ec21d9adf1b7.umls_semantic_type_tree.tsv


In [3]:

# Function by github.com/lubianat with some slight alterations by github.com/jvfe
@lru_cache(maxsize=None)
def get_wikidata_item(wikidata_property, value):
    """Gets a Wikidata item for a determined property-value pair


      Args:
        property (str): The property to search
        value (str): The value of said property
      
      Returns:
        str: A Wikidata item or a "None" value if no item found.
    """
    query_result = wdi_core.WDItemEngine.execute_sparql_query(
        f'SELECT distinct ?item WHERE {{ ?item wdt:{wikidata_property} "{value}" }}'
    )
    try:
        match = query_result["results"]["bindings"][0]
    except:
        return None
    qid = match["item"]["value"]

    qid = qid.split("/")[4]
    return qid

In [29]:
def get_wdt_items_from_umls_entities(doc):
    """Create a table from the UMLS entities and link them to WDT
    """
    identified = []
    for ent in doc.ents:
        print(ent)
        try:
            best_id = ent._.kb_ents[0][0]
        except IndexError:
            best_id = None
        print(best_id)
        identified.append([ent.text, ent.start_char, ent.end_char, best_id])

    entity_df = pd.DataFrame.from_records(identified, 
                                        columns=['label', 'start_pos', 'end_pos', 'umls_id'])
    
    entity_df['qid'] = entity_df['umls_id'].apply(lambda x: get_wikidata_item("P2892", x))

    return entity_df

In [30]:
text = """
Name-calling in the hippocampus (and beyond): coming to terms with neuron types and properties
D."""

doc = nlp(text)

/home/lubianat/.local/lib/python3.6/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/lubianat/.local/lib/python3.6/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [31]:
doc.ents

(Name-calling, hippocampus, neuron types, properties)

In [32]:
get_wdt_items_from_umls_entities(doc)

Name-calling
None
hippocampus
C0019564
neuron types
C0332307
properties
C0871161


,label,start_pos,end_pos,umls_id,qid
0,Name-calling,1,13,None,None
1,hippocampus,21,32,C0019564,None
2,neuron types,68,80,C0332307,Q104370009
3,properties,85,95,C0871161,None
